## This is script is used for program custom generators


In [2]:
# imports 
import tensorflow as tf
from tensorflow.keras.utils import Sequence
from glob import glob
import numpy as np
import math
!pip install pydicom
import pydicom

In [3]:
# define generators from inheritting Sequence class
class DicomGenegeratorAutoTFio(Sequence):
    
    # default functions needs to be overwriiten
    def __init__(self, batch_size, train_or_test, dims=(512, 512),shuffle=True, train_images_root=None, val_images_root=None, n_channels=1):
        self.train_images_root = train_images_root
        self.val_images_root = val_images_root
        self.batch_size = batch_size
        self.train_or_test =  train_or_test
        self.dims = dims
        self.shuffle = shuffle
        self.n_channels = n_channels
       
        
        # custom functions inside function---------->
        def get_data_paths(root_dir):
            all_paths = [] 
            for each in root_dir:
                print("root:", each)
                one_root_paths =  sorted(glob(each +'/*.DCM'))
                print("one_root_paths:", len(one_root_paths))
                print()
                all_paths = all_paths+ one_root_paths
            return all_paths

        if self.train_or_test == "train":
            self.dcm_paths = get_data_paths(self.train_images_root)
#             print(f'Found {len(self.dcm_paths)} training images')
        else: 
            self.dcm_paths =  get_data_paths(self.val_images_root)
#             print(f'Found {len(self.dcm_paths)} validation images')

        self.on_epoch_end()
        

    def __len__(self):
        num_batches = math.ceil(len(self.dcm_paths) / self.batch_size)
        print(f'Found {len(self.dcm_paths)} {self.train_or_test} images ')
        return math.ceil(len(self.dcm_paths) / self.batch_size)

    def __getitem__(self, idx):
        # Generate indexes of the batch
        indexes = self.indexes[idx*self.batch_size: (idx+1)*self.batch_size]
        
        # Find related DCMs' paths
        batch_temp_dcm_paths = [self.dcm_paths[k] for k in indexes]
        
      
        return self._generate_X_X(batch_temp_dcm_paths)
    
    def _generate_X_X(self, batch_temp_dcm_paths):
        """
        batch_temp_dcm_paths : the batch of paths for reading dcms
        return: return the numpy array of dcm raw pix data
        """
        # initialization
        X = np.empty((self.batch_size, *self.dims, self.n_channels))
#         y = np.empty((self.batch_size), dtype=int)
        # Generate data
        for i, path in enumerate(batch_temp_dcm_paths):
            # Store sample
            temp = self._load_dcm(path)
            X[i,] = np.expand_dims(temp, axis=-1)
        
        return X, X
    
    def _generate_Y(self, batch_temp_dcm_paths):
        pass
    
    def _load_dcm(self,dcm_path):
        print(dcm_path)
        ds = pydicom.dcmread(dcm_path)
        pix = ds.pixel_array
        print("Shape info: ---------->", pix.shape )
        print("raw DCM content range [{}, {}]".format(np.min(pix), np.max(pix)))
        
    def on_epoch_end(self):
        #'Updates indexes after each epoch'
        self.indexes = np.arange(len(self.dcm_paths))
        if self.shuffle == True:
            np.random.shuffle(self.indexes)

## main test

In [ ]:

if __name__ == '__main__':
    # root dirs for dcms
    train_images_root = sorted(glob('/media/ytx/Japan_Deep_Data/dataset/LeiSang/myTry/BleedingDataDCM/train/*'))
    val_images_root = sorted(glob('/media/ytx/Japan_Deep_Data/dataset/LeiSang/myTry/BleedingDataDCM/val/*'))
    
    #hyperparameters
    batch_size = 2
    
    # create generator instances
    train_dcm_gen =  DicomGenegeratorAutoTFio(batch_size=batch_size, 
                                          train_or_test="train", 
                                          shuffle=False,
                                          train_images_root=train_images_root)
    val_dcm_gen =  DicomGenegeratorAutoTFio(batch_size=batch_size, 
                                          train_or_test="val", 
                                          shuffle=False,
                                          val_images_root=val_images_root)
    
    print(f'Found {train_dcm_gen.__len__()} training images')
    print(f'Found {val_dcm_gen.__len__()} validation images')
    
    # check the data in data generator
    for idx, data in enumerate(train_dcm_gen):
        print(idx)
        print("train_sample shape:", data[0].shape) # remember data generator now return (X, X)
        print("train_target shape:", data[1].shape)
        print("min: {} max:{}".format(np.min(data), np.max(data)))
        
    for idx, data in enumerate(val_dcm_gen):
        print(idx)
        print("train_sample shape:", data[0].shape) # remember data generator now return (X, X)
        print("train_target shape:", data[1].shape)
        print("min: {} max:{}".format(np.min(data), np.max(data)))

    
    print("train dataset is ok")
    print("val dataset is ok")

root: /media/ytx/Japan_Deep_Data/dataset/LeiSang/myTry/BleedingDataDCM/train/ZA-006_000
one_root_paths: 1251

root: /media/ytx/Japan_Deep_Data/dataset/LeiSang/myTry/BleedingDataDCM/train/ZA-008_000
one_root_paths: 1182

root: /media/ytx/Japan_Deep_Data/dataset/LeiSang/myTry/BleedingDataDCM/train/ZA-012_000
one_root_paths: 665

root: /media/ytx/Japan_Deep_Data/dataset/LeiSang/myTry/BleedingDataDCM/train/ZA-015_001
one_root_paths: 1132

root: /media/ytx/Japan_Deep_Data/dataset/LeiSang/myTry/BleedingDataDCM/train/ZA-017_001
one_root_paths: 1226

root: /media/ytx/Japan_Deep_Data/dataset/LeiSang/myTry/BleedingDataDCM/train/ZA-019_001
one_root_paths: 1176

root: /media/ytx/Japan_Deep_Data/dataset/LeiSang/myTry/BleedingDataDCM/train/ZA-020_002
one_root_paths: 1251

root: /media/ytx/Japan_Deep_Data/dataset/LeiSang/myTry/BleedingDataDCM/train/ZA-021_001
one_root_paths: 1219

root: /media/ytx/Japan_Deep_Data/dataset/LeiSang/myTry/BleedingDataDCM/train/ZA-022_001
one_root_paths: 1251

root: /medi